# 评价特征
+	重要性（ 出度中心性、介数中心性、紧密中心性）

这个参考14论文

+	所在领域竞争力（所在领域的艺术家人数，所在领域人数越多，影响力越大）

举例说明：同一水平的英文作家的影响力肯定比祖鲁语作家的影响力高，可以理解为作品类型本身的传播能力

+	所在领域年份人数

举例说明：百家争鸣时代的普通天才往往能在星光黯淡的时代大放异彩，可以理解为艺术在所在领域在艺术家活跃期间获取影响力的难度

+	所在领域年份影响力排名（先筛选出艺术家所在领域此年份的所有人，然后根据影响人数进行排名）（排名越小，影响力越大）

很直观的指标

+	所在年份竞争力（这一年份的所有领域的艺术家总人数）（竞争力越大，影响力越大）

艺术家瓜分影响力这块蛋糕的难度（45用相关性分析后续再决定选择）

+	当前年代追随者数目

直观

+	（非当前年代追随者数目）

艺术家在创作生涯结束后对未来时代的影响力衡量

+	追随者跨的领域数目

艺术家作品对非本领域艺术家的吸引程度，也可以说是艺术家作品所含的纯粹的艺术性的度量

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import string
import pandas as pd 
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import networkx as nx  


In [2]:
data=pd.read_csv('./2021_ICM_Problem_D_Data/influence_data.csv',encoding='gbk')

In [3]:
data.head()

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
0,759491,The Exploited,Pop/Rock,1980,74,Special Duties,Pop/Rock,1980
1,25462,Tricky,Electronic,1990,335,PJ Harvey,Pop/Rock,1990
2,66915,Bob Dylan,Pop/Rock,1960,335,PJ Harvey,Pop/Rock,1990
3,71209,Leonard Cohen,Pop/Rock,1950,335,PJ Harvey,Pop/Rock,1990
4,91438,The Gun Club,Pop/Rock,1980,335,PJ Harvey,Pop/Rock,1990


# 异常值检测和处理

In [11]:
data[data['influencer_active_start']>data['follower_active_start']].head()#不将这些influencer比follewer晚的作为异常值
#由此可以证明艺术家创作活跃时间有限

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
190,145443,Fito Olivares,Latin,1990,4852,Selena,Latin,1980
320,752945,Paul van Dyk,Electronic,1990,6232,Robert Miles,Electronic,1980
358,172642,Joe Mooney,Vocal,1950,6334,Tony Bennett,Vocal,1940
362,212612,Johnny Hartman,Vocal,1950,6334,Tony Bennett,Vocal,1940
390,307135,Rick Nielsen,Pop/Rock,1970,7230,Brian May,Pop/Rock,1960


# 制作networkx 有向图对象

In [4]:
G=nx.DiGraph()

In [5]:
influencer=[]
follower=[]
for line in data.iterrows():
    influencer.append(line[1]['influencer_id'])
    follower.append(line[1]['follower_id'])


In [6]:
for i in range(len(influencer)):
    G.add_edge(influencer[i],follower[i])#此处有向图的指向是1指向2

In [12]:
nodes=G.node()

# 异常值处理2
+ id为[893383, 303506, 957340]的influencer出现了统计错误，分别把他们的一个follower作为id对应的influncer的名字和年份了

我们将这类错误行全部删除

In [19]:
error_list
#id=893383的全删除，因为两只乐队都不多且混杂严重
#删除id=303506，姓名为Day26的一行（因为其它24行是usher）

[893383, 303506, 957340]

In [27]:
data[data['influencer_id']==893383]

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
6450,893383,The Muffs,Pop/Rock,1990,97200,Bracket,Pop/Rock,1990
6644,893383,The Kills,Pop/Rock,1990,99508,The Kills,Pop/Rock,2000
18669,893383,The Muffs,Pop/Rock,1990,329232,New Found Glory,Pop/Rock,1990
20046,893383,The Muffs,Pop/Rock,1990,366681,Paramore,Pop/Rock,2000
26355,893383,The Muffs,Pop/Rock,1990,585574,Sum 41,Pop/Rock,1990
33376,893383,The Muffs,Pop/Rock,1990,784535,The Donnas,Pop/Rock,1990
41095,893383,The Muffs,Pop/Rock,1990,1013109,Hey Monday,Pop/Rock,2000


In [34]:
data.drop([6644,29381,37697],inplace=True)

# factor1 重要度计算

In [35]:
deg=nx.out_degree_centrality(G)#出度中心性

In [36]:
bet=nx.betweenness_centrality(G)#介数中心性

In [37]:
clo=nx.closeness_centrality(G)#紧密度中心性

In [48]:
maxclo_key=0
maxclo_value=0
for i in clo:
    if clo[i]>maxclo_value:
        maxclo_value=clo[i]
        maxclo_key=i


In [49]:
maxbet_key=0
maxbet_value=0
for i in bet:
    if bet[i]>maxbet_value:
        maxbet_value=bet[i]
        maxbet_key=i


In [50]:
maxdeg_key=0
maxdeg_value=0
for i in deg:
    if deg[i]>maxdeg_value:
        maxdeg_value=bet[i]
        maxdeg_key=i


In [56]:
#每个人的向量
arrays={}
for node in nodes:
    array=[]
    array.append(deg[node]/maxdeg_value)
    array.append(bet[node]/maxbet_value)
    array.append(clo[node]/maxclo_value)
    arrays[node]=array

In [57]:
idea_array=[1,1,0]

In [58]:
#算距离
distances={}
for array in arrays:
    distance=np.square(arrays[array][0]-idea_array[0])+np.square(arrays[array][1]-idea_array[1])+np.square(arrays[array][2]-idea_array[2])
    distance=np.sqrt(distance)
    distances[array]=distance